In [32]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os


data_path = 'dataset/'

In [33]:
# Transform
data_transforms = {
    'test':
        transforms.Compose([
            transforms.ToTensor(),
            # from Preprocess file result
            transforms.Normalize([0.4388, 0.4213, 0.4016], [0.2508, 0.2558, 0.2859])
        ]),
}

In [34]:
# Data load and basic info
# Data load
image_datasets = {x: datasets.ImageFolder(os.path.join(data_path, x),data_transforms[x])
                  for x in ['test']}

# Into dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)
              for x in ['test']}

# basic info
dataset_sizes = {x: len(image_datasets[x]) for x in ['test']}
class_names = image_datasets['test'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(class_names)
print(dataset_sizes)
print(device)

['altar', 'apse', 'bell_tower', 'column', 'dome(inner)', 'dome(outer)', 'flying_buttress', 'gargoyle', 'stained_glass', 'vault']
{'test': 1404}
cpu


In [35]:
# Reconstruct Model
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))
model_ft = model_ft.to(device)
model_ft.load_state_dict(torch.load('model.pth'))

criterion = nn.CrossEntropyLoss()

In [36]:
def test_model(model, criterion, test_dir):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders[test_dir]):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
        loss = running_loss / dataset_sizes[test_dir]
        acc = running_corrects.double() / dataset_sizes[test_dir]
        
        print(f'{test_dir} Loss: {loss:.4f} Acc: {acc:.4f}')

In [37]:
test_model(model_ft, criterion, 'test')

test Loss: 0.3411 Acc: 0.9017
